# 🌍 ESG & Financial Intelligence Platform
## AI-Powered Analysis of Corporate Reports

**🎯 What this does:**
- Automatically processes PDF reports from Google Cloud Storage
- Uses Gemini 2.5 Pro to extract financial & ESG metrics
- Generates forecasts using Google's TimesFM 2.0 model
- Analyzes companies: Amgen, Novartis, Target

**📊 Features:**
- ✅ Automated PDF processing from Cloud Storage
- ✅ AI-powered data extraction (Financial + ESG metrics)
- ✅ Revenue forecasting with TimesFM 2.0
- ✅ Multi-company comparative analysis

---
**👆 Click "Runtime" → "Run all" to see the magic happen!**

In [1]:
# 🚀 Setup & Authentication
print("🌟 Welcome to the ESG & Financial Intelligence Platform!")
print("🔧 Setting up environment...")

# Authenticate with Google Cloud
from google.colab import auth
auth.authenticate_user()

# Install required packages
!pip install -q bigframes google-cloud-bigquery plotly seaborn

# Import libraries
import bigframes.pandas as bpd
from google.cloud import bigquery
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt

print("✅ Authentication complete!")
print("✅ Libraries installed!")

🌟 Welcome to the ESG & Financial Intelligence Platform!
🔧 Setting up environment...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.7 MB/s eta 0:00:00
✅ Authentication complete!
✅ Libraries installed!


In [2]:
# 🏗️ Project Configuration
from google.colab import userdata
import getpass

# Get project configuration
try:
    PROJECT_ID = userdata.get('GOOGLE_CLOUD_PROJECT')
    print(f"✅ Using project from secrets: {PROJECT_ID}")
except:
    PROJECT_ID = getpass.getpass("🔑 Enter your Google Cloud Project ID: ")

# Configure BigFrames
bpd.options.bigquery.project = PROJECT_ID
bpd.options.bigquery.location = "US"

# Initialize BigQuery client
client = bigquery.Client(project=PROJECT_ID)

print(f"✅ Project configured: {PROJECT_ID}")
print("✅ BigFrames ready for action!")

🔑 Enter your Google Cloud Project ID: ··········
✅ Project configured: intellitrend-project-dev
✅ BigFrames ready for action!


In [5]:
# 📁 Step 0: Create External Table for PDF Reports
print("📁 Creating external table for PDF reports...")

create_external_table_sql = f"""
CREATE OR REPLACE EXTERNAL TABLE `{PROJECT_ID}.db_reports_insights.all_reports`
WITH CONNECTION `{PROJECT_ID}.us.aiconn`
OPTIONS (
  object_metadata = 'SIMPLE',
  uris = ['gs://report_insights/*']
);
"""

try:
    job = client.query(create_external_table_sql)
    job.result()
    print("✅ External table created successfully!")

    # Check what files we have
    check_files_sql = f"SELECT uri, size FROM `{PROJECT_ID}.db_reports_insights.all_reports` LIMIT 10"
    files_df = bpd.read_gbq(check_files_sql)
    print(f"📄 Found {len(files_df)} files in storage")
    print("Sample files:")
    print(files_df.head())

except Exception as e:
    print(f"⚠️  Note: {e}")
    print("   Make sure your GCS bucket and connection are set up correctly")

📁 Creating external table for PDF reports...
✅ External table created successfully!


📄 Found 10 files in storage
Sample files:


                                                 uri      size
0  gs://report_insights/amgen/amgen_annualreport_...   8154936
1  gs://report_insights/amgen/amgen_annualreport_...   3651904
2  gs://report_insights/amgen/amgen_sustainabilit...   4345660
3  gs://report_insights/amgen/amgen_sustainabilit...   3692343
4  gs://report_insights/amgen/amgen_annualreport_...  12725236

[5 rows x 2 columns]
